In [1]:
from gurobipy import Model, GRB, quicksum
import numpy as np
from itertools import product

In [2]:
from itertools import product

num_weeks = 9
num_consmatches_1 = 4
num_consmatches_2 = 5

def valid_pattern(pattern):
    h_count = pattern.count('H')
    a_count = pattern.count('A')
    
    # Check the total H/A counts
    if not (
        (h_count == num_consmatches_1 and a_count == num_consmatches_2) or
        (h_count == num_consmatches_2 and a_count == num_consmatches_1)
    ):
        return False
    
    # Check for "breaks" only between odd/even round pairs
    for i in range(0, len(pattern), 2):
        if i + 1 < len(pattern):
            if pattern[i] == pattern[i + 1]:
                return False
    
    return True

all_patterns = product('HA', repeat=num_weeks)
valid_patterns = [
    ''.join(p) for p in all_patterns
    if valid_pattern(p)
]

num_patterns = len(valid_patterns)
print(f"Total valid patterns: {len(valid_patterns)}")
print(valid_patterns)


Total valid patterns: 32
['HAHAHAHAH', 'HAHAHAHAA', 'HAHAHAAHH', 'HAHAHAAHA', 'HAHAAHHAH', 'HAHAAHHAA', 'HAHAAHAHH', 'HAHAAHAHA', 'HAAHHAHAH', 'HAAHHAHAA', 'HAAHHAAHH', 'HAAHHAAHA', 'HAAHAHHAH', 'HAAHAHHAA', 'HAAHAHAHH', 'HAAHAHAHA', 'AHHAHAHAH', 'AHHAHAHAA', 'AHHAHAAHH', 'AHHAHAAHA', 'AHHAAHHAH', 'AHHAAHHAA', 'AHHAAHAHH', 'AHHAAHAHA', 'AHAHHAHAH', 'AHAHHAHAA', 'AHAHHAAHH', 'AHAHHAAHA', 'AHAHAHHAH', 'AHAHAHHAA', 'AHAHAHAHH', 'AHAHAHAHA']


In [3]:
# Dictionary `feasible_opponents` where feasible_opponents[i, t]
# is the set of feasible opponents for pattern i in week t


# Initialize the feasible opponents dictionary
feasible_opponents = {}

# Iterate through each pattern and week
num_patterns = len(valid_patterns)
num_weeks = len(valid_patterns[0])

for i in range(num_patterns):
    for t in range(num_weeks):
        # Get the H/A status of pattern i in week t
        current_status = valid_patterns[i][t]
        
        # Find all patterns that have the opposite status in week t
        feasible_opponents[i, t] = {j for j in range(num_patterns) if j != i and valid_patterns[j][t] != current_status}


# Print the feasible opponents dictionary for verification
for key, value in feasible_opponents.items():
    print(f"Pattern {key[0]} in week {key[1]} can play against patterns {value}")


Pattern 0 in week 0 can play against patterns {16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 1 can play against patterns {16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 2 can play against patterns {8, 9, 10, 11, 12, 13, 14, 15, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 3 can play against patterns {8, 9, 10, 11, 12, 13, 14, 15, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 4 can play against patterns {4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31}
Pattern 0 in week 5 can play against patterns {4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31}
Pattern 0 in week 6 can play against patterns {2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31}
Pattern 0 in week 7 can play against patterns {2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31}
Pattern 0 in week 8 can play against patterns {1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31}
Pattern 1 in week 0 can pl

In [4]:

from gurobipy import Model, GRB, quicksum

# Initialize m
m = Model("Sports_Scheduling")

## Parameters

# S = set of patterns
S = range(len(valid_patterns))

# T = set of rounds - weeks
T = range(num_weeks - 1)

# M = number of teams to be scheduled (here 10)
M = 10

# Omega_{s,t} = set of feasible opponents for pattern s in round t
# Rename 'feasible_opponents' dictionary to Omega:
Omega = feasible_opponents 

# v_{s,s',t} = 1 if pattern s is paired with pattern s' in round t
v = m.addVars(S, S, T, vtype=GRB.BINARY, name="v")

# u_s = 1 if pattern s is selected
u = m.addVars(S, vtype=GRB.BINARY, name="u")

# (1b) Exactly M patterns must be selected:
m.addConstr(quicksum(u[s] for s in S) == M, name="(1b)_select_M_patterns")

# (1c) If pattern s is used in round t, it must pair up with exactly one opponent in Omega_{s,t}.
#      Summation over feasible opponents = u_s
for s in S:
    for t_ in T:
        # If s is in the domain of Omega and has feasible opponents:
        if (s, t_) in Omega:
            m.addConstr(
                quicksum(v[s, s_prime, t_] 
                         for s_prime in Omega[s, t_] if s_prime > s) +
                quicksum(v[s_prime, s, t_] 
                         for s_prime in Omega[s, t_] if s_prime < s)
                == u[s],
                name=f"(1c)_consistency_s{s}_t{t_}"
            )

# Also, if s' is not in Omega_{s,t}, set an upper bound of 0:
for s in S:
    for t_ in T:
        for s_prime in S:
            if s_prime not in Omega.get((s, t_), []):
                # Already set to 0 if s >= s_prime, so check if s < s_prime
                if s < s_prime:
                    v[s, s_prime, t_].ub = 0


# (1d) Each pair (s, s') can appear in at most one round t, 
#      but only if s is selected:
for s in S:
    for s_prime in S:
        if s != s_prime:
            m.addConstr(
                quicksum(v[s, s_prime, t_] for t_ in T) <= u[s],
                name=f"(1d)_unique_match_s{s}_sprime{s_prime}"
            )


Set parameter Username


Academic license - for non-commercial use only - expires 2025-11-04


In [5]:
m.optimize()

if m.status == GRB.OPTIMAL:
    # v_{s,s',t} = 1 if the pattern pair (s, s') is scheduled in round t
    v_sol = {
        (s, s_prime, t_): v[s, s_prime, t_].X
        for s in S
        for s_prime in S
        for t_ in T
        if v[s, s_prime, t_].X > 0.5
    }

    # u_s = 1 if pattern s is selected
    u_sol = {
        s: u[s].X
        for s in S
        if u[s].X > 0.5
    }

    solution_found = {
        "v": v_sol,
        "u": u_sol
    }
    print("Optimal solution found.")
    print("Selected patterns (u_s = 1):", u_sol)
    print("Match assignments (v_{s,s',t} = 1):", v_sol)
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1249 rows, 8224 columns and 13312 nonzeros
Model fingerprint: 0x00b3d7ef
Variable types: 0 continuous, 8224 integer (8224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]


Presolve removed 512 rows and 6144 columns
Presolve time: 0.04s
Presolved: 737 rows, 2080 columns, 6912 nonzeros
Variable types: 0 continuous, 2080 integer (2080 binary)

Root relaxation: objective 0.000000e+00, 701 iterations, 0.06 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   94          -    0.00000      -     -    0s
     0     0    0.00000    0  165          -    0.00000      -     -    0s
H    0     0                       0.0000000    0.00000  0.00%     -    0s

Cutting planes:
  Gomory: 19
  Clique: 22
  Zero half: 11

Explored 1 nodes (5982 simplex iterations) in 0.74 seconds (0.28 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Optimal solution found.
Selected 

In [6]:
import pandas as pd

# Extract weeks dynamically
weeks = sorted({t for _, _, t in v_sol.keys()})
calendar_data = {f"Week {t+1}": [] for t in weeks}

# Populate calendar data for each week 
for t in weeks:
    week_pairs = []
    for (i, j, w) in v_sol:
        if w == t:
            week_pairs.append(f"Pattern {i} vs Pattern {j}")
    calendar_data[f"Week {t+1}"] = week_pairs

# Create and display the DataFrame
calendar_df = pd.DataFrame(calendar_data)
calendar_df

,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8
0,Pattern 2 vs Pattern 29,Pattern 2 vs Pattern 31,Pattern 2 vs Pattern 25,Pattern 2 vs Pattern 27,Pattern 2 vs Pattern 4,Pattern 2 vs Pattern 6,Pattern 2 vs Pattern 13,Pattern 2 vs Pattern 16
1,Pattern 3 vs Pattern 16,Pattern 3 vs Pattern 27,Pattern 3 vs Pattern 31,Pattern 3 vs Pattern 25,Pattern 3 vs Pattern 6,Pattern 3 vs Pattern 29,Pattern 3 vs Pattern 4,Pattern 3 vs Pattern 13
2,Pattern 4 vs Pattern 27,Pattern 4 vs Pattern 16,Pattern 4 vs Pattern 29,Pattern 4 vs Pattern 13,Pattern 13 vs Pattern 16,Pattern 4 vs Pattern 25,Pattern 6 vs Pattern 16,Pattern 4 vs Pattern 6
3,Pattern 6 vs Pattern 25,Pattern 6 vs Pattern 29,Pattern 6 vs Pattern 13,Pattern 6 vs Pattern 31,Pattern 25 vs Pattern 29,Pattern 13 vs Pattern 27,Pattern 25 vs Pattern 31,Pattern 25 vs Pattern 27
4,Pattern 13 vs Pattern 31,Pattern 13 vs Pattern 25,Pattern 16 vs Pattern 27,Pattern 16 vs Pattern 29,Pattern 27 vs Pattern 31,Pattern 16 vs Pattern 31,Pattern 27 vs Pattern 29,Pattern 29 vs Pattern 31


In [9]:
for i in solution_found['u']:
    print(f"Pattern {i}: {valid_patterns[i]}")

Pattern 2: HAHAHAAHH
Pattern 3: HAHAHAAHA
Pattern 4: HAHAAHHAH
Pattern 6: HAHAAHAHH
Pattern 13: HAAHAHHAA
Pattern 16: AHHAHAHAH
Pattern 25: AHAHHAHAA
Pattern 27: AHAHHAAHA
Pattern 29: AHAHAHHAA
Pattern 31: AHAHAHAHA


In [10]:
solution_found

{'v': {(2, 4, 4): 1.0,
  (2, 6, 5): 1.0,
  (2, 13, 6): 1.0,
  (2, 16, 7): 1.0,
  (2, 25, 2): 1.0,
  (2, 27, 3): 1.0,
  (2, 29, 0): 1.0,
  (2, 31, 1): 1.0,
  (3, 4, 6): 1.0,
  (3, 6, 4): 1.0,
  (3, 13, 7): 1.0,
  (3, 16, 0): 1.0,
  (3, 25, 3): 1.0,
  (3, 27, 1): 1.0,
  (3, 29, 5): 1.0,
  (3, 31, 2): 1.0,
  (4, 6, 7): 1.0,
  (4, 13, 3): 1.0,
  (4, 16, 1): 1.0,
  (4, 25, 5): 1.0,
  (4, 27, 0): 1.0,
  (4, 29, 2): 1.0,
  (6, 13, 2): 1.0,
  (6, 16, 6): 1.0,
  (6, 25, 0): 1.0,
  (6, 29, 1): 1.0,
  (6, 31, 3): 1.0,
  (13, 16, 4): 1.0,
  (13, 25, 1): 1.0,
  (13, 27, 5): 1.0,
  (13, 31, 0): 1.0,
  (16, 27, 2): 1.0,
  (16, 29, 3): 1.0,
  (16, 31, 5): 1.0,
  (25, 27, 7): 1.0,
  (25, 29, 4): 1.0,
  (25, 31, 6): 1.0,
  (27, 29, 6): 1.0,
  (27, 31, 4): 1.0,
  (29, 31, 7): 1.0},
 'u': {2: 1.0,
  3: 1.0,
  4: 1.0,
  6: 1.0,
  13: 1.0,
  16: 1.0,
  25: 1.0,
  27: 1.0,
  29: 1.0,
  31: 1.0}}